In [1]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timezone
import scipy as sp
from os import times
import time
from turtle import color

# Black Scholes Equation:

$$\frac{\partial V}{\partial t}
+ \frac{1}{2}\sigma^{2} S^{2} \frac{\partial^{2} V}{\partial S^{2}}
+ r S \frac{\partial V}{\partial S}
- r V = 0$$

<br>

**Time derivatives**

$\frac{\partial V}{\partial t}$ rate of change of option value with respect to time:  
$$\frac{V_{i,k} - V_{i,k+1}}{\Delta t}$$


$\frac{\partial^{2} V}{\partial S^{2}}$ second derivative with respect to stock price:
$$\frac{V_i+1,_k - 2V_i,_k  + V_i-1_,k}{\Delta S^2}$$

<br>


$\frac{\partial V}{\partial S}$ = first derivative with respect to stock price:
$$\frac{V_{i+1,k} - V_{i-1,k}}{2\Delta S}$$

**Explicit FDM**
- Starting with the BSE, substitue in the derivatives
$$\frac{\partial V}{\partial t}
+ \frac{1}{2}\sigma^{2} S^{2} \frac{\partial^{2} V}{\partial S^{2}}
+ r S \frac{\partial V}{\partial S}
- r V = 0$$


$$\frac{V_{i,k} - V_{i,k+1}}{\Delta t} + \frac{1}{2} \sigma^2 S_i^2 \frac{V_i+1,_k - 2V_i,_k  + V_i-1_,k}{\Delta S^2} + rS_i \frac{V_{i+1,k} - V_{i-1,k}}{2\Delta S} - rV_{i,k} = 0 $$

<br>
Move the first derivative to the lhs to solve for $V_{i,k}$

$$V_{i,k}  = V_{i,k+1} +  \Delta t + \frac{1}{2} \sigma^2 S_i^2 \frac{V_{i+1,k+1} - 2V{_i,_{k+1}}  + V_{i-1_,k+1}}{\Delta S^2} + rS_i \frac{V_{i+1,k+1} - V_{i-1,k+1}}{2\Delta S} - rV_{i,k+1}$$

<br>
Collect the coefficients:

  $$V_{s,t} = A_t V_{s+1,t-1} + B_t V_{s+1, t} + C_t V_{s+1,t+1}$$
<br>


where:

$$A_t = \frac{1}{2} \Delta t \left(\sigma^2 i^2 - r i\right)$$

$$B_t = 1 - \Delta t \left(\sigma^2 i^2 + r\right)$$

$$C_t = \frac{1}{2} \Delta t \left(\sigma^2 i^2 + r i\right)$$

The option price today is $V_{s, 0}$ where $s$ corresponds to the current stock price


**Boundary Conditions for European Calls**:
<br>
$$V(0,t) = 0 {\forall i}$$

$$V(S,T) = \max(S - K,0) $$

$$V(S_{max},t)=S_{max} - Ke^{−r(T−t)}$$
<br>




In [2]:
def number_of_days_asset_level(time_lvl, asset_level):
  return np.zeros((time_lvl + 1,asset_level + 1), dtype=np.float64)

In [3]:
def set_boundary_condition_bottom(MeshGrid):
  for i in range(MeshGrid.shape[0]):
    MeshGrid[i,0] = 0
  return MeshGrid

In [4]:
def set_terminal_condition(MeshGrid, spatial_step, K, Max):
  for i in range(MeshGrid.shape[1]):
    MeshGrid[Max, i] = max(spatial_step * i - K, 0)
  return MeshGrid

In [5]:
def set_right_boundary_condition(MeshGrid, K, Smax,t, T, r):
  for k in range(MeshGrid.shape[0]):
    MeshGrid[k, -1] = Smax - K * np.exp(-r * (T - t[k]))
  return MeshGrid


In [6]:

def bse_exp_call(sigma, risk_free_rate, time_to_exp, K, N, spatial_step, S0):
  start = time.time()
  #Choosing a Smax for the right boundary condition
  Smax = max(3 * K, 2.5 * S0)

  #Choosing the stock price step
  M = int(Smax/spatial_step)
  delta_t = time_to_exp/N #time_step

  t = np.linspace(0, time_to_exp, N + 1) #Time axis N+1 points from t=0 to t=T
  S = np.linspace(0, Smax, M + 1) # Stock price axis M+1 points from S=0 S=Smax

  # Setting up the boundary and terminal conditions in our grid
  Grid = number_of_days_asset_level(N, M)
  Grid = set_boundary_condition_bottom(Grid)
  Grid = set_terminal_condition(Grid,spatial_step, K, N)
  Grid = set_right_boundary_condition(Grid, K, Smax, t, time_to_exp, risk_free_rate )

  A = np.zeros(M+1, dtype=np.float64)
  B = np.zeros(M+1,dtype=np.float64)
  C = np.zeros(M+1, dtype=np.float64)

  #Calculate A_ki B_ki C_ki
  for j in range(1,M):
        A[j] = 0.5 * delta_t * (sigma ** 2 * j ** 2 - risk_free_rate * j)
        B[j] = 1 - delta_t * (sigma ** 2 * j ** 2 + risk_free_rate)
        C[j] = 0.5 * delta_t * (sigma ** 2 * j ** 2 + risk_free_rate * j)

  for k in range(N-1, -1, -1):
      for i in range(1, M):
          Grid[k, i] = (
              A[i] * Grid[k+1, i-1] +
              B[i] * Grid[k+1, i] +
              C[i] * Grid[k+1, i+1]
          )
  end = time.time()
  time_taken = end - start
  return Grid, t,S, time_taken

In [7]:
def get_contract_information(contract, ctrct_type="call"):
  aapl = yf.Ticker("AAPL")
  data = aapl.history()
  close = data['Close'].iloc[-1]

  opt = yf.Ticker(contract)
  info = opt.info

  option_price = info["regularMarketPrice"] 

  expiry = datetime.fromisoformat(info["expireIsoDate"].replace("Z", "+00:00"))
  now = datetime.now(timezone.utc)
  time_to_exp = (expiry - now).total_seconds() / (365 * 24 * 3600)
  strike_K = info['strike']
  if ctrct_type == "call":
    sigma =  imp_vol(0.0375,time_to_exp,strike_K,close,option_price,"call")
  else:
    sigma =  imp_vol(0.0375,time_to_exp,strike_K,close,option_price,"put")


  risk_free_rate = 0.0735 #US 3m bond rate
  print(f"Time to expiry years", time_to_exp)
  print(f"Option Price", option_price)
  print(f"Close:", close)
  print(f"Strike: ${info['strike']:.2f}")
  print("=====")
  print(f"Vol/Sigma: ",sigma)

  return risk_free_rate, time_to_exp, strike_K, close


In [8]:
get_contract_information("AAPL260109C00267500")

NameError: name 'imp_vol' is not defined

In [ ]:
def plot_bse(Grid, T, S):
  T_grid, S_grid = np.meshgrid(T, S, indexing="ij")
  fig = plt.figure(figsize=(10, 6))
  ax = fig.add_subplot(111, projection="3d")

  ax.plot_surface(
      S_grid,    # x-axis: underlying price
      T_grid,    # y-axis: time
      Grid,      # z-axis: option value
      linewidth=0,
      antialiased=True
  )

  ax.set_xlabel("Underlying price S")
  ax.set_ylabel("Time t")
  ax.set_zlabel("Option value V(t, S)")
  ax.set_title("Explicit finite-difference Black–Scholes (Call)")

  plt.show()

In [10]:
risk_free_rate, time_to_exp, strike_K, close = get_contract_information("AAPL260109C00267500")

NameError: name 'imp_vol' is not defined

In [9]:
Grid, T, S, t = bse_exp_call(0.2440057306393271, risk_free_rate, time_to_exp, strike_K, 25000, 0.75, close)

plot_bse(Grid, T, S)

spl = sp.interpolate.CubicSpline(S, Grid[0, :])

print(" Explicit Price: ",float (spl(271.01)))
print("Analytical Price: ",bse_analytical(sigma, risk_free_rate, time_to_exp, strike_K, close, option_type="call"))

NameError: name 'risk_free_rate' is not defined

# Put Option
Boundary Conditions for European Puts:
$$V(0,t) = K e^{−r(T−t)}$$
$$V(S,T) = \max(K - S,0) $$
$$V(S_{max},t)=0 {\forall i} $$




In [11]:
def set_left_boundary_condition_put(MeshGrid, K, Smax,t, T, r):
  for k in range(MeshGrid.shape[0]):
    MeshGrid[k, 0] = K * np.exp(-r * (T - t[k]))
  return MeshGrid

In [12]:
def set_boundary_condition_right_put(MeshGrid):
  for i in range(MeshGrid.shape[0]):
    MeshGrid[i,-1] = 0
  return MeshGrid

In [13]:
def set_terminal_condition_put(MeshGrid, timeStep, K, Max):
  for i in range(MeshGrid.shape[1]):
    MeshGrid[Max, i] = max(K - (timeStep * i) , 0)
  return MeshGrid

In [14]:
def number_of_days_asset_level(time_lvl, asset_level):
  return np.zeros((time_lvl + 1,asset_level + 1), dtype=np.float64)

In [15]:
def bse_exp_put(sigma, risk_free_rate, time_to_exp, K, N, spatial_step, S0):
  start = time.time()
  Smax = max(3 * K, 2.5 * S0)

  M = int(Smax/spatial_step)
  delta_t = time_to_exp/N #delta T
  t = np.linspace(0, time_to_exp, N + 1) #Time axis N+1 points from t=0 to t=T
  S = np.linspace(0, Smax, M + 1) # Stock price axis M+1 points from S=0 S=Smax

  # Setting up the boundaries where the value of the option will be V(t_k, S_i)
  Grid = number_of_days_asset_level(N, M)
  Grid = set_terminal_condition_put(Grid,spatial_step, K, N)

  Grid = set_left_boundary_condition_put(Grid, K, Smax, t, time_to_exp, risk_free_rate )
  Grid = set_boundary_condition_right_put(Grid)
  #print(Grid)

  A = np.zeros(M+1, dtype=np.float64)
  B = np.zeros(M+1,dtype=np.float64)
  C = np.zeros(M+1, dtype=np.float64)
  #Calculate A_ki B_ki C_ki

  for j in range(1,M):
        A[j] = 0.5 * delta_t * (sigma ** 2 * j ** 2 - risk_free_rate * j)
        B[j] = 1 - delta_t * (sigma ** 2 * j ** 2 + risk_free_rate)
        C[j] = 0.5 * delta_t * (sigma ** 2 * j ** 2 + risk_free_rate * j)

  for k in range(N-1, -1, -1):
      for i in range(1, M):
          Grid[k, i] = (
              A[i] * Grid[k+1, i-1] +
              B[i] * Grid[k+1, i] +
              C[i] * Grid[k+1, i+1]
          )
  end = time.time()
  time_taken = end - start
  return Grid, t,S, time_taken

In [16]:
risk_free_rate, time_to_exp, strike_K, close = get_contract_information("AAPL260116P00267500","put")
Grid, T, S, t = bse_exp_put(0.21410444843974563, risk_free_rate, time_to_exp, strike_K, 25000, 0.75, close)

plot_bse(Grid, T, S)

spl = sp.interpolate.CubicSpline(S, Grid[0, :])

print(" Explicit Price: ",float (spl(271.01)))
print("Analytical Price: ",bse_analytical(0.21410444843974563, risk_free_rate, time_to_exp, strike_K, close, option_type="put"))



NameError: name 'imp_vol' is not defined

# European Down and Out


*   At expiry pays the same as a call given that the price of the asset hasn't fallen below the barrier.
*   If the price of the asset falls below the barrier at any time before expiry then the option will be worthless.
* Should cost less than a corresponding call, due to the added risk of being knocked out


Boundary Conditions:
$$V_{Do}(S,T) = max(S-K, 0)$$
$$V_{Do}(S_{max},t)=S_{max} - Ke^{−r(T−t)}$$
$$V_{Do}(S = {barrier},t) = 0 \forall T $$

In [17]:
def bse_exp_down_out(sigma, risk_free_rate, time_to_exp, K, N, spatial_step, S0, barrier):
  start = time.time()
  Smax = max(3 * K, 2.5 * S0)

  M = int(Smax/spatial_step)
  delta_t = time_to_exp/N #delta T
  t = np.linspace(0, time_to_exp, N + 1) #Time axis N+1 points from t=0 to t=T
  S = np.linspace(barrier, Smax, M + 1) # Stock price axis M+1 points from S=0 S=Smax

  dS = (Smax - barrier)/M

  # Setting up the boundaries where the value of the option will be V(t_k, S_i)
  Grid = number_of_days_asset_level(N, M)
  Grid = set_boundary_condition_bottom(Grid)
  Grid[N, :] = np.maximum(S - K, 0.0)
  Grid[:, 0] = 0.0

  Grid = set_right_boundary_condition(Grid, K, Smax, t, time_to_exp, risk_free_rate )

  A = np.zeros(M+1, dtype=np.float64)
  B = np.zeros(M+1,dtype=np.float64)
  C = np.zeros(M+1, dtype=np.float64)
  #Calculate A_ki B_ki C_ki

  for j in range(1,M):
        S_j = barrier + j * dS
        A[j] = 0.5 * delta_t * (sigma**2 * S_j**2 / dS**2 - risk_free_rate * S_j / dS)
        B[j] = 1 - delta_t * (sigma**2 * S_j**2 / dS**2 + risk_free_rate)
        C[j] = 0.5 * delta_t * (sigma**2 * S_j**2 / dS**2 + risk_free_rate * S_j / dS)

  for k in range(N-1, -1, -1):
      for i in range(1, M):
          Grid[k, i] = (
              A[i] * Grid[k+1, i-1] +
              B[i] * Grid[k+1, i] +
              C[i] * Grid[k+1, i+1]
          )
  end = time.time()
  time_taken = end - start
  return Grid, t,S, time_taken

In [ ]:
Grid, T, S,t = bse_exp_down_out(0.1730,0.0375,0.0548,275.00,25000,0.75, 274.80,270)
spl = sp.interpolate.CubicSpline(S, Grid[0, :])

print( "SPL",float (spl(274.80)))
plot_bse(Grid, T, S)

# Performance Testing


In [ ]:
time_taken = np.zeros(7)

S0 = 274.80
K  = 275.00
T  = 0.0548
sigma = 0.1730
r = 0.0375

N = 25000
dS_list = [ 3, 1.5, 0.75, 0.375]

times = []
for dS in dS_list:
    Grid, t, S, tt = bse_exp_call(sigma, r, T, K, N, dS, S0)
    times.append(float(tt))  # ensure scalar

plt.plot(dS_list, times, marker="o")
plt.xlabel("Spatial step ΔS")
plt.ylabel("Time taken (seconds)")
plt.title("Runtime vs spatial step (ΔS)")
plt.gca().invert_xaxis()
plt.grid(True)
plt.show()



In [ ]:

S0 = 274.80
K  = 275.00
T  = 0.0548
sigma = 0.1730
r = 0.0375

N = 25000
dS_list = np.array([3, 1.5, 0.75, 0.375], dtype=float)

prices = []
for dS in dS_list:
    Grid, t, S, tt = bse_exp_call(sigma, r, T, K, N, dS, S0)
    spl = sp.interpolate.CubicSpline(S, Grid[0, :])
    price = float(spl(S0))
    prices.append(price)
    print(dS, price)

prices = np.array(prices)
bs = float(bse_analytical(sigma, r, T, K, S0))

errors = prices - bs
abs_errors = np.abs(errors)

plt.figure(figsize=(8, 5))

plt.plot(dS_list, prices, marker="o", linewidth=2, label="Explicit FDM price")

plt.axhline(bs, linestyle="--", linewidth=2, label="Analytical (Black–Scholes)")

plt.gca().invert_xaxis()

plt.xlabel("Spatial step size ΔS (smaller = finer grid)")
plt.ylabel("Call option price ($)")
plt.title("Explicit FDM convergence to Black–Scholes price")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print("BS:", bs)
print("errors:", errors.tolist())


# American Options
# Call

In [ ]:

def bse_exp_call_us(sigma, risk_free_rate, time_to_exp, K, N, spatial_step, S0):
  start = time.time()
  #Choosing a Smax for the right boundary condition
  Smax = max(3 * K, 2.5 * S0)

  #Choosing the stock price step
  M = int(Smax/spatial_step)
  delta_t = time_to_exp/N #time_step

  t = np.linspace(0, time_to_exp, N + 1) #Time axis N+1 points from t=0 to t=T
  S = np.linspace(0, Smax, M + 1) # Stock price axis M+1 points from S=0 S=Smax
  Payoff = Grid[-1, :]

  # Setting up the boundary and terminal conditions in our grid
  Grid = number_of_days_asset_level(N, M)
  Grid = set_boundary_condition_bottom(Grid)
  Grid = set_terminal_condition(Grid,spatial_step, K, N)
  Grid = set_right_boundary_condition(Grid, K, Smax, t, time_to_exp, risk_free_rate )

  A = np.zeros(M+1, dtype=np.float64)
  B = np.zeros(M+1,dtype=np.float64)
  C = np.zeros(M+1, dtype=np.float64)

  #Calculate A_ki B_ki C_ki
  for j in range(1,M):
        A[j] = 0.5 * delta_t * (sigma ** 2 * j ** 2 - risk_free_rate * j)
        B[j] = 1 - delta_t * (sigma ** 2 * j ** 2 + risk_free_rate)
        C[j] = 0.5 * delta_t * (sigma ** 2 * j ** 2 + risk_free_rate * j)

  for k in range(N-1, -1, -1):
      for i in range(1, M):
          Grid[k, i] = (
              A[i] * Grid[k+1, i-1] +
              B[i] * Grid[k+1, i] +
              C[i] * Grid[k+1, i+1]
          )
          Grid[k,i] = max(Grid[k,i], Payoff[i])

  end = time.time()
  time_taken = end - start
  return Grid, t,S, time_taken

In [ ]:
risk_free_rate, time_to_exp, strike_K, close = get_contract_information("AAPL260109C00267500")
Grid, T, S, t = bse_exp_call(0.2440057306393271, risk_free_rate, time_to_exp, strike_K, 25000, 0.75, close)
Grid, T, S, t = bse_exp_call_us(0.2440057306393271, risk_free_rate, time_to_exp, strike_K, 25000, 0.75, close)

plot_bse(Grid, T, S)

spl = sp.interpolate.CubicSpline(S, Grid[0, :])

print(" Explicit Price: ",float (spl(271.01)))
print("Analytical Price: ",bse_analytical(sigma, risk_free_rate, time_to_exp, strike_K, close, option_type="call"))

# Put

In [ ]:
def bse_exp_put_us(sigma, risk_free_rate, time_to_exp, K, N, spatial_step, S0):
  start = time.time()
  Smax = max(3 * K, 2.5 * S0)

  M = int(Smax/spatial_step)
  delta_t = time_to_exp/N #delta T
  t = np.linspace(0, time_to_exp, N + 1) #Time axis N+1 points from t=0 to t=T
  S = np.linspace(0, Smax, M + 1) # Stock price axis M+1 points from S=0 S=Smax


  # Setting up the boundaries where the value of the option will be V(t_k, S_i)
  Grid = number_of_days_asset_level(N, M)
  Grid = set_terminal_condition_put(Grid,spatial_step, K, N)

  Grid = set_left_boundary_condition_put(Grid, K, Smax, t, time_to_exp, risk_free_rate )
  Grid = set_boundary_condition_right_put(Grid)
  #print(Grid)
  Payoff = np.maximum(K - S, 0)  # Put payoff: max(Strike - Stock, 0)


  A = np.zeros(M+1, dtype=np.float64)
  B = np.zeros(M+1,dtype=np.float64)
  C = np.zeros(M+1, dtype=np.float64)
  #Calculate A_ki B_ki C_ki

  for j in range(1,M):
        A[j] = 0.5 * delta_t * (sigma ** 2 * j ** 2 - risk_free_rate * j)
        B[j] = 1 - delta_t * (sigma ** 2 * j ** 2 + risk_free_rate)
        C[j] = 0.5 * delta_t * (sigma ** 2 * j ** 2 + risk_free_rate * j)

  for k in range(N-1, -1, -1):
      for i in range(1, M):
          Grid[k, i] = (
              A[i] * Grid[k+1, i-1] +
              B[i] * Grid[k+1, i] +
              C[i] * Grid[k+1, i+1]
          )
          Grid[k,i] = max(Grid[k,i], Payoff[i])

  end = time.time()
  time_taken = end - start
  return Grid, t,S, time_taken

In [ ]:
risk_free_rate, time_to_exp, strike_K, close = get_contract_information("AAPL260116P00267500","put")
Grid, T, S, t = bse_exp_put(0.21410444843974563, risk_free_rate, time_to_exp, strike_K, 25000, 0.75, close)
spl = sp.interpolate.CubicSpline(S, Grid[0, :])
print(" Explicit Price EUR: ",float (spl(271.01)))

Grid, T, S, t = bse_exp_put_us(0.21410444843974563, risk_free_rate, time_to_exp, strike_K, 25000, 0.75, close)

plot_bse(Grid, T, S)

spl = sp.interpolate.CubicSpline(S, Grid[0, :])

print(" Explicit Price USA: ",float (spl(271.01)))
print("Analytical Price: ",bse_analytical(0.21410444843974563, risk_free_rate, time_to_exp, strike_K, close, option_type="put"))

